# **CNN assignment**(Do read the note at the end)

Description: Here we are going to use CNN to classify images of elephants,tigers,cheetah's and crocodiles. Basically multiclass classificiation using CNN.

**Q.1** Import all required modules

In [52]:
#Your code here
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

seed=0
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

**Q.2** Load data using image_dataset_from_directory(https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory)

In [27]:
# Creating a Dataset
data_dir = '/content/data'

data= tf.keras.utils.image_dataset_from_directory(
    data_dir,  # Directory where the Training images are located
    labels = 'inferred', # Classes will be inferred according to the structure of the directory
    label_mode = 'categorical',
    class_names = ['Cheetah', 'Crocodile', 'Elephants', 'Tiger'],
    batch_size = 16,    # Number of processed samples before updating the model's weights
    image_size = (256, 256), # Defining a fixed dimension for all images
    shuffle = True,  # Shuffling data
    seed = seed,  # Random seed for shuffling and transformations
    validation_split = 0, # We don't need to create a validation set from the training set
    crop_to_aspect_ratio = True # Resize images without aspect ratio distortion
)

Found 1658 files belonging to 4 classes.




*   Since data object is not iterable,we will create a nummpy iterator for data.
*   Then use batch to iterate through our data



In [28]:
data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [29]:
data_iterator=data.as_numpy_iterator()

In [30]:
data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [15]:
batch=data_iterator.next()

Run the below code and batch code 2-3 times to visualise the labels given to different animals.

In [19]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
  ax[idx].imshow(img.astype(int))
  ax[idx].title.set_text(batch[1][idx].any)

**Q.3** Map the data such that all pixel values lie between 0 and 1 using the **lambda function**

In [21]:
scaler = tf.keras.layers.Rescaling(1./255) # Defining scaler values between 0 to 1
data=data.map(lambda x, y: (scaler(x), y))

In [26]:
data

<_MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

**Q.4** Split the data into train set and test set(and validation set if needed) using **data.take**(https://www.geeksforgeeks.org/tensorflow-js-tf-data-dataset-class-take-method/) and **data.skip**(https://www.geeksforgeeks.org/tensorflow-js-tf-data-dataset-skip-method/)

In [35]:
#Your code here
# Determine split sizes
total_size = len(data)
train_size = int(0.7 * total_size)  # 70% of total
test_size = int(0.2 * total_size)   # 20% of total
validation_size = total_size - train_size - test_size  # Remaining for validation

# Split the dataset
train_data = data.take(train_size)
test_data = data.skip(train_size).take(test_size)
validation_data = data.skip(train_size + test_size).take(validation_size)

**Creating the model**

**Q.5** Write code to create CNN model including fully connected layers with softmax as final layer.

In [87]:
#model=#Your code here

# Initiating model on GPU
#with tf.device("/cpu:0"):
model = Sequential()

#model.add(augmentation) # Adding data augmentation pipeline to the model

# Feature Learning Layers
model.add(Conv2D(32,                  # Number of filters/Kernels
                  (3,3),               # Size of kernels (3x3 matrix)
                  strides = 1,         # Step size for sliding the kernel across the input (1 pixel at a time).
                  padding = 'same',    # 'Same' ensures that the output feature map has the same dimensions as the input by padding zeros around the input.
                input_shape = (256,256,3) # Input image shape
                ))
model.add(Activation('relu'))# Activation function
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5,5), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv2D(256, (5,5), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dropout(0.3))

# Flattening tensors
model.add(Flatten())

# Fully-Connected Layers
model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(4, activation = 'softmax')) # Classification layer

**Q.6** Compile your model

In [88]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(0.0001), # 1e-4
              loss = 'categorical_crossentropy', # Ideal for multiclass tasks
              metrics = ['accuracy']) # Evaluation metric

**Q.7** Fit your model with train set and make sure to keep **less epochs**(10-15) as the dataset size is very large.

In [95]:
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('best_model.h5',
                            monitor = 'val_accuracy',
                            save_best_only = True)

model.fit(train_data, epochs=100)   #steps_per_epoch=10

Epoch 1/100
72/72 [==============================] - 11s 136ms/step - loss: 1.6990 - accuracy: 0.7170
Epoch 2/100
72/72 [==============================] - 11s 143ms/step - loss: 1.6363 - accuracy: 0.7387
Epoch 3/100
72/72 [==============================] - 12s 144ms/step - loss: 1.3200 - accuracy: 0.7708
Epoch 4/100
72/72 [==============================] - 12s 144ms/step - loss: 0.9338 - accuracy: 0.8229
Epoch 5/100
72/72 [==============================] - 11s 130ms/step - loss: 0.8303 - accuracy: 0.8420
Epoch 6/100
72/72 [==============================] - 11s 126ms/step - loss: 0.7519 - accuracy: 0.8611
Epoch 7/100
72/72 [==============================] - 11s 136ms/step - loss: 0.5955 - accuracy: 0.8750
Epoch 8/100
72/72 [==============================] - 12s 156ms/step - loss: 0.4707 - accuracy: 0.9045
Epoch 9/100
72/72 [==============================] - 12s 143ms/step - loss: 0.5687 - accuracy: 0.9036
Epoch 10/100
72/72 [==============================] - 11s 137ms/step - loss: 0.517

In [96]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 256, 256, 32)      896       
                                                                 
 activation_42 (Activation)  (None, 256, 256, 32)      0         
                                                                 
 batch_normalization_35 (Ba  (None, 256, 256, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 128, 128, 32)      0         
 ng2D)                                                           
                                                                 
 dropout_42 (Dropout)        (None, 128, 128, 32)      0         
                                                                 
 conv2d_36 (Conv2D)          (None, 128, 128, 64)    

**Q.8** Test your code with test set

In [102]:
# Loading best weights
#model.load_weights('best_model.h5')

preds = model.predict(test_data)  # Running model on the validation dataset
loss,accuracy = model.evaluate(test_data) # Obtaining Loss and Accuracy on the val dataset

print('\nTest Loss: ', loss)
print('\nTest Accuracy: ', np.round(accuracy * 100), '%')

#model.save('plant_disease_classifier.h5') # Saving model

20/20 [==============================] - 10s 63ms/step - loss: 1.4219 - accuracy: 0.8844

Test Loss:  1.4219143390655518

Test Accuracy:  88.0 %


In [103]:
# Do not change this code
if accuracy>=0.85:
  print(f"Congratulations, CNN assignment complete!! Your accuracy is {accuracy}")
else:
  print(f"Try again, not enough accuracy! Your accuracy is {accuracy}" )

Congratulations, CNN assignment complete!! Your accuracy is 0.8843749761581421


**Note: 1. Many of you will get very high train accuracy but low test accuracy. This is called overfitting, this can be solved by increasing the train set size. If your getting low accuracy for both train and test, modify your layers in the model.**

**2.Do use T4GPU.Ideally your first epoch will take quite sometime and rest of the epochs will be faster but if all your epochs are taking a lot of time, try starting a new gpu session(open a new gmail account and open colab on that) because your free gpu might have exhausted itself.**

**3.Other than that everything should be fine, happy learning!!**